In [4]:
import thingspeak
import os
import requests
import pandas as pd
import json
from datetime import datetime
import openpyxl
import urllib
from concurrent.futures import ThreadPoolExecutor, as_completed
import aqi

# Sets working directory to package directory.
os.chdir('/mnt/c/Users/iozeroff/OneDrive - Earthwatch/Desktop/Data-Science/Python-Projects/oha-ups/')

In [ ]:
# To Do:
    # Timezone offsets.
    # Calculating the Purple Air EPA score from the concentration numbers.
    # Complete all docstrings.
    # Clean up all XXX's.

In [5]:
def get_thingspeak_keys(sensors):
    """Get channelID and read key from PurpleAir API for a list of sensorID's."""
    sensor_dict = {}
    API_root = "https://www.purpleair.com/json?"
    # Asserts entered argument is of type list.
    # XXX This will be an issue if passing in a Pandas Series.
    assert isinstance(sensors, list), "Sensors should be a list of 5-digit integers."
    # Asserts length of all values in sensors list are of length 5.
    assert any([len(str(x)) == 5 for x in sensors]), "One or more sensors IDs are not 5-digits"
    with ThreadPoolExecutor(max_workers=2) as executor:
        for i in sensors:
            response = requests.get(API_root, params={'show':i})
            assert response.status_code == 200, print(response.status_code)    
            # Pulls results dictionary from response JSON.    
            results = response.json().get('results')
            # Defines channel ids and read keys for primary and secondary channels.
            device_name = results[0].get('Label')
            channelA_id = results[0].get('THINGSPEAK_PRIMARY_ID')
            channelA_key = results[0].get('THINGSPEAK_PRIMARY_ID_READ_KEY')
            channelB_id = results[1].get('THINGSPEAK_PRIMARY_ID')
            channelB_key = results[1].get('THINGSPEAK_PRIMARY_ID_READ_KEY')
            # Writes primary and secondary channel id's and read keys into dictionary.
            sensor_dict[device_name] = {'A':(channelA_id, channelA_key), 'B':(channelB_id, channelB_key)}
    return(sensor_dict)

In [10]:
def get_thingspeak(channel, **kwargs):
    """Submit GET request to thingspeak API using specified arguments.
    
    :param channel: An object of class thingspeak.channel.
    :param \**kwargs: See below

    :Keyword Arguments:
       start (string): Character string of date in ISO 8601 format to supply to requests.get options argument as start date for channel data.
       end (string): Character string of date in ISO 8601 format to supply to requests.get options argument as end date for channel data.
       days (int): XXX Fill
       average (int): XXX Fill
    """
    # This block confirms all arguments are acceptable. 
    # Thingspeak API may still reject inputs if it doesn't recognize channelID or read_key.
    # Figure out TimeZone offset (daylight savings changes the offset!).
    expected_args = ('start', 'end', 'days', 'average')
    assert any([i in expected_args for i in kwargs.keys()]), "received an unexpected argument. See documentation for list of expected arguments."
    if not ('average' in kwargs.keys()):
        # Appends default argument average=10 to kwargs if not present.
        kwargs.update({'average':10})
    try:
        response = json.loads(channel.get(options=kwargs))
    except urllib.error.HTTPError as error:
        print(error)
    else:
        # Creates fields object to use for column naming.
        fields = response['channel']
        # Creates data frame from response JSON dictionary.
        df = pd.DataFrame.from_dict(response['feeds']).set_index(['created_at']).rename(columns=fields)
        return(df)

In [7]:
# Defines write_to_excel function, that writes input pandas DataFrame to excel or csv file.
# XXX Clean pathname. Ideally would use Sensor label to name excel file. Also add hyphens.
def write_to_excel(channel_A, channel_B, filename):
    """Write inputted dataframe to either csv (default) or excel file."""
    d = datetime.today()
    assert isinstance(channel_A, pd.DataFrame), "channel_A should be a pandas DataFrame."
    assert isinstance(channel_B, pd.DataFrame), "channel_A should be a pandas DataFrame."
    assert isinstance(filename, str), "Filename passed to write_df is not a character string."
    pathname = 'outputs/datasets/' + filename + "-" + str(d.strftime('%Y%m%d'))
    full_path = pathname + ".xlsx"
    with pd.ExcelWriter(full_path) as writer:  # doctest: +SKIP
        channel_A.to_excel(writer, sheet_name='Channel A')
        channel_B.to_excel(writer, sheet_name='Channel B')
    
    

In [ ]:
sensor = [29709]
start = "2019-10-01T00:00:00" # If either of the dates are set as None, datetime.strptime fails.
end = "2019-10-23T23:59:59"
# Number of 24-hour periods before now to include in response. The default is 1.
days=None
# Number of 60-second periods before now to include in response. The default is 1440.
minutes=None

channel_id = 742875
read_key = 'JC4FCUJOPST94F65'
channel = thingspeak.Channel(id=channel_id, api_key=read_key)
df = get_thingspeak(channel, start=start, end=end)

In [25]:
sensor = [29023, 29669]
get_thingspeak_keys(sensor)

# XXX Functionalize
# XXX Find way to pass down **kwargs.
sensor_dict = get_thingspeak_keys([29709])
start = "2019-10-01 00:00:00" 
end = "2019-10-01 06:59:59"
with ThreadPoolExecutor(max_workers=2) as executor:
    for sensor in sensor_dict:
        sensor_df_dict = {}
        for i in sensor_dict[sensor]: 
            channel_id = sensor_dict[sensor][i][0]
            read_key = sensor_dict[sensor][i][1]
            # Establishes thingspeak channel object (connection).
            channel = thingspeak.Channel(id=channel_id, api_key=read_key)
            # Submits GET request to thingspeak API using pre-specified arguments.
            df = get_thingspeak(channel, start=start, end=end)
            sensor_df_dict[i] = df
        unused_colsA = ['Uptime', 'RSSI']
        unused_colsB = ['Mem', 'Adc', 'Unused']
        sensor_df_dict['A'] = sensor_df_dict['A'].drop(unused_colsA, axis='columns')
        sensor_df_dict['B'] = sensor_df_dict['B'].drop(unused_colsB, axis='columns')
        filename = sensor
        write_to_excel(sensor_df_dict['A'], sensor_df_dict['B'], filename)
    

In [36]:
response = json.loads(channel.get(options={'start':start, 'end':end}))
fields = response['channel']
print(fields)

{'Red Acre Stow': {'A': ('742875', 'JC4FCUJOPST94F65'), 'B': ('742877', '169K93UX17ZDR12C')}}


HTTPError: 400 Client Error: Bad Request for url: https://api.thingspeak.com/channels/742875/feeds.json?end=2019-10-23T23%3A59%3A59&api_key=JC4FCUJOPST94F6&start=2019-10-01T00%3A00%3A00

In [48]:
# Add Docstring.
def get_purple_air(sensors, **kwargs):
    """"""
    sensor_dict = get_thingspeak_keys(sensors)
    for sensor in sensor_dict:
        for i in sensor 
            channel_id = sensor_dict[sensor][i][0]
            read_key = sensor_dict[sensor][i][1]
            # Establishes thingspeak channel object (connection).
            channel = thingspeak.Channel(id=channel_id, api_key=read_key)
            # Submits GET request to thingspeak API using pre-specified arguments.
            # XXX Error being thrown from **kwargs here.
            response = get_thingspeak(channel, **kwargs)
            # Creates fields object to use for column naming.
            fields = response['channel']
            # Creates data frame from response JSON dictionary.
            df = pd.DataFrame.from_dict(response['feeds']).set_index('created_at')
            # Creates dataframe from json response. 
            df = df.rename(columns=fields).drop(['RSSI', 'Uptime'], axis=1)
            filename = str(sensor_dict[i])

In [23]:
response = get_thingspeak(channel, start=start, end=end, average=10)
print(response.info())
# Creates fields object to use for column naming.

# Creates fields object to use for column naming.
fields = response['channel']
print(fields)
# Creates data frame from response JSON dictionary.
df = pd.DataFrame.from_dict(response['feeds']).set_index('created_at').rename(columns=fields)
print(df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 1606 entries, 2019-10-12T20:20:00Z to 2019-10-23T23:50:00Z
Data columns (total 8 columns):
0.3um            1603 non-null object
0.5um            1603 non-null object
1.0um            1603 non-null object
2.5um            1603 non-null object
5.0um            1603 non-null object
10.0um           1603 non-null object
PM1.0 (CF=1)     1603 non-null object
PM10.0 (CF=1)    1603 non-null object
dtypes: object(8)
memory usage: 112.9+ KB
None


In [19]:
channel = thingspeak.Channel(id=channel_id, api_key=read_key)
start = "2019-10-01T00:00:00" # If either of the dates are set as None, datetime.strptime fails.
end = "2019-10-23T23:59:59"
# Number of 24-hour periods before now to include in response. The default is 1.
days=None
# Number of 60-second periods before now to include in response. The default is 1440.
minutes=None
# Get average of this many minutes, valid values: 10, 15, 20, 30, 60, 240, 720, 1440, "daily".
average=None
# Submits GET request to thingspeak API using pre-specified arguments.
# XXX Error being thrown from **kwargs here.
response = get_thingspeak(channel, start=start, end=end)
print(response)

                      entry_id   0.3um   0.5um  1.0um 2.5um 5.0um 10.0um  \
created_at                                                                 
2019-10-12T20:22:39Z    122046  228.61   66.74   2.93  0.00  0.00   0.00   
2019-10-12T20:24:39Z    122047  249.77   63.23   7.49  1.31  0.00   0.00   
2019-10-12T20:26:39Z    122048  260.00   67.59   9.33  0.46  0.00   0.00   
2019-10-12T20:28:39Z    122049  296.74   84.75   6.06  1.06  0.00   0.00   
2019-10-12T20:30:39Z    122050  303.04   84.56   6.77  1.83  0.43   0.43   
2019-10-12T20:32:39Z    122051  240.31   65.28   8.09  1.58  0.84   0.45   
2019-10-12T20:34:39Z    122052  257.61   70.06   6.61  0.41  0.00   0.00   
2019-10-12T20:36:39Z    122053  237.09   65.62   8.26  0.00  0.00   0.00   
2019-10-12T20:38:39Z    122054  310.48   84.86   8.09  1.45  0.96   0.96   
2019-10-12T20:40:39Z    122055  248.43   70.75   5.00  0.97  0.50   0.00   
2019-10-12T20:42:39Z    122056  284.52   77.59   6.55  0.00  0.00   0.00   
2019-10-12T2